In [57]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
scaler = StandardScaler()
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.model_selection import cross_val_score, cross_val_predict

###


from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 


In [58]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file
def checkDir(dirLoc):
    print
    if os.path.exists(dirLoc):#Checks if the dir exists
        print("The directory exists")
    else:
        print ("No directory found for "+dirLoc) #Output if no directory
#         print
#         os.makedirs(CheckDir)#Creates a new dir for the given name
#         print "Directory created for "+CheckDir

In [21]:
hardDrivesLoc = '/media/ak/'
hardDrivesList = os.listdir('/media/ak/')

dataOnlyDrive = '/media/ak/DataOnly'  # external date only drive
#ext_drive_loc = ('/media/ak/My Passport/Experiment Data/')
data_dir = os.getenv('FINANCE_DATA') #internal folder with finance data 
folderList =  [s for s in os.listdir(dataOnlyDrive) if s.startswith('Dat') or s.startswith('Fin')]
# inputDrive = data_only_drive
# features = os.path.join(inputDrive,'features')
# labels = os.path.join(inputDrive, 'labels')

# symbols = [s for s in os.listdir(features) if s.endswith('.L')]
folderIdx = 0
finalLocation= "/".join((dataOnlyDrive,folderList[folderIdx]))
symbols = [s for s in os.listdir(finalLocation) if s.endswith('.L')] #keep a list of the symbols
os.listdir(finalLocation)

['CNA.L', 'PRU.L', 'Labels', 'REL.L', 'BARC.L']

In [30]:
locIdx= 1

selection = hardDrivesList[locIdx]
selectionLoc = os.path.join(hardDrivesLoc,selection )
os.listdir(selectionLoc)
dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
DataLoc = os.path.join(hardDrivesLoc,selection,dataList[1] )
path= 'MKL_Experiments'


In [35]:
#os.makedirs(os.path.join(DataLoc,path)) # run once

In [34]:
MKLExpPath = os.path.join(DataLoc,path)
MKLExpPath
checkDir(MKLExpPath)


The directory exists


## pick a symbol ## 

In [36]:
##picking up a specific feature
symbolIdx = 1 #pick one of the symbols
#symbols[symbolIdx] -->output :PRU.L
print(symbols[symbolIdx])
# do a join to get the location
symbolLocation = "/".join((finalLocation,symbols[symbolIdx])) 

# get he features now
symbolFeaturesLocation = "/".join((symbolLocation,'MODEL_BASED')) # where all the HMM output is

print(symbolFeaturesLocation) # <-- all the HMM model output is here, for each model there is a Date Folder and then OOS files

PRU.L
/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED


In [41]:
#os.makedirs(os.path.join(MKLExpPath,symbols[symbolIdx])) #make MKL path for symbol
MKLSymbolPath = os.path.join(MKLExpPath,symbols[symbolIdx])

### pick Features for a specific HMM MODEL Date ###
### then move to labels ...pick the location of the labels and make a list of all the available labels ##

In [37]:

symbolHMMDatesList = os.listdir(symbolFeaturesLocation) 

#list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
## (symbolHMMDatesList = '20170829', '20170710', '20170801', ... ]
#location of labels : /media/ak/DataOnly/FinDataReal/Labels/[Symbol :PRU.L]/NON_DIRECTIONAL

symbolLabelsLocation = "/".join((finalLocation, 'Labels',symbols[symbolIdx],'NON_DIRECTIONAL')) 

#list of all the label dates

symbolLabelsDates =[dateFile.split(".")[0] for dateFile in os.listdir(symbolLabelsLocation)]

# Output symbolLabelsDates --> ['20170704', '20180226', '20180208',...] all we are doing is going for this 
symbolFeaturesLocation


'/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED'

In [7]:
##now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
hmmFeatureLocations ={} #symbol-hmm-model-date index --> this is the indexation in symbolFeaturesDatesList
commonDatesDict ={} # this is a struct that will contain for each HMM date, the common labels/features- this should be used for training and testing
createDate =[] #place holder for the hash key of when the features got created
commonFeaturesLocs ={}
commonLabelsLocs ={}
commonLocs ={}

#this symbolFeaturesDatesList[featrsIdx] will give you a date: 20170710 =which contains all the HMM induced featureb
for hmmDateIdx, hmmDate in enumerate(np.sort(symbolHMMDatesList)):
    
    symbolModelFeaturesDate = "/".join((symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx]))
    createDate = os.listdir(symbolModelFeaturesDate)[2].split("_")[7] #stupid hack

    #output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710
    symbolEachModelFeaturesDates=[file.split("_")[5] for file in os.listdir(symbolModelFeaturesDate )] 
    #output is a list of dates

    commonDates =list(set(symbolEachModelFeaturesDates) & set(symbolLabelsDates)) # this is a list of features and labels dates
    
    for commonDate in np.sort(commonDates):
        print("create Date:###",createDate)
        print("common Date:-----", commonDate)
        labelsCommonDateFile = ".".join((commonDate,'csv'))
        labelsCommonFileLoc = "/".join((symbolLabelsLocation, labelsCommonDateFile))
        labelsDf=pd.read_csv(labelsCommonFileLoc)
        comnDateFeatureLocMaster =("/".join((symbolFeaturesLocation, hmmDate)))
        commonDatesFeatureFile = "".join((symbols[1],'_3_states_features_date:_',commonDate,"_now:_",createDate,"_.pickle"))
        FeatureFileLoc = "/".join((comnDateFeatureLocMaster,commonDatesFeatureFile))
       
        commonLocs[commonDate]=[FeatureFileLoc,labelsCommonFileLoc ]
        print(commonLocs[commonDate])



create Date:### 20181229
common Date:----- 20170830
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170116/PRU.L_3_states_features_date:_20170830_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170830.csv']
create Date:### 20181229
common Date:----- 20170831
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170116/PRU.L_3_states_features_date:_20170831_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170831.csv']
create Date:### 20181229
common Date:----- 20180201
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170116/PRU.L_3_states_features_date:_20180201_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180201.csv']
create Date:### 20181229
common Date:----- 20180202
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170116/PRU.L_3_states_features_date:_20180202_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180202.cs

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20170712_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170712.csv']
create Date:### 20181229
common Date:----- 20170713
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20170713_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170713.csv']
create Date:### 20181229
common Date:----- 20170714
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20170714_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170714.csv']
create Date:### 20181229
common Date:----- 20170717
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20170717_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170717.csv']
create Date:### 20181229
common Date:----- 20170

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20180208_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180208.csv']
create Date:### 20181229
common Date:----- 20180209
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20180209_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180209.csv']
create Date:### 20181229
common Date:----- 20180212
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20180212_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180212.csv']
create Date:### 20181229
common Date:----- 20180213
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20180213_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180213.csv']
create Date:### 20181229
common Date:----- 20180

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170118/PRU.L_3_states_features_date:_20180220_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180220.csv']
create Date:### 20181229
common Date:----- 20180221
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170118/PRU.L_3_states_features_date:_20180221_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180221.csv']
create Date:### 20181229
common Date:----- 20180222
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170118/PRU.L_3_states_features_date:_20180222_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180222.csv']
create Date:### 20181229
common Date:----- 20180223
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170118/PRU.L_3_states_features_date:_20180223_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180223.csv']
create Date:### 20181229
common Date:----- 20180

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170119/PRU.L_3_states_features_date:_20170725_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170725.csv']
create Date:### 20181229
common Date:----- 20170726
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170119/PRU.L_3_states_features_date:_20170726_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170726.csv']
create Date:### 20181229
common Date:----- 20170727
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170119/PRU.L_3_states_features_date:_20170727_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170727.csv']
create Date:### 20181229
common Date:----- 20170728
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170119/PRU.L_3_states_features_date:_20170728_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170728.csv']
create Date:### 20181229
common Date:----- 20170

...

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170721/PRU.L_3_states_features_date:_20180412_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180412.csv']
create Date:### 20181229
common Date:----- 20180413
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170721/PRU.L_3_states_features_date:_20180413_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180413.csv']
create Date:### 20181229
common Date:----- 20180416
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170721/PRU.L_3_states_features_date:_20180416_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180416.csv']
create Date:### 20181229
common Date:----- 20180417
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170721/PRU.L_3_states_features_date:_20180417_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180417.csv']
create Date:### 20181229
common Date:----- 20180

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170726/PRU.L_3_states_features_date:_20180403_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180403.csv']
create Date:### 20181229
common Date:----- 20180404
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170726/PRU.L_3_states_features_date:_20180404_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180404.csv']
create Date:### 20181229
common Date:----- 20180405
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170726/PRU.L_3_states_features_date:_20180405_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180405.csv']
create Date:### 20181229
common Date:----- 20180406
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170726/PRU.L_3_states_features_date:_20180406_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180406.csv']
create Date:### 20181229
common Date:----- 20180

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170727/PRU.L_3_states_features_date:_20180201_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180201.csv']
create Date:### 20181229
common Date:----- 20180202
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170727/PRU.L_3_states_features_date:_20180202_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180202.csv']
create Date:### 20181229
common Date:----- 20180205
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170727/PRU.L_3_states_features_date:_20180205_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180205.csv']
create Date:### 20181229
common Date:----- 20180206
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170727/PRU.L_3_states_features_date:_20180206_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180206.csv']
create Date:### 20181229
common Date:----- 20180

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170728/PRU.L_3_states_features_date:_20180413_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180413.csv']
create Date:### 20181229
common Date:----- 20180416
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170728/PRU.L_3_states_features_date:_20180416_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180416.csv']
create Date:### 20181229
common Date:----- 20180417
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170728/PRU.L_3_states_features_date:_20180417_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180417.csv']
create Date:### 20181229
common Date:----- 20180418
['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170728/PRU.L_3_states_features_date:_20180418_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180418.csv']
create Date:### 20181229
common Date:----- 20180

IndexError: list index out of range

In [42]:
#do this to store commonLocs in the appropriate folder
commonLocsSymbolPath = os.path.join(MKLSymbolPath, 'commonLocs.pickle')
with open(commonLocsSymbolPath, 'wb') as handle:
    pickle.dump(commonLocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
with open(commonLocsSymbolPath, 'rb') as handle:
    b = pickle.load(handle)
commonLocs == b # simple check- delete

True

###Functions for Training Set ###

In [96]:
# all the keys the model fitting key - can use something like this to test?!


In [97]:
def forwardDates(idxKey, commonLocs):
    ''' return all the forward looking dates for each idxKey we use for training'''
    keys =list(commonLocs.keys())
    lookAheadKeys = sorted(i for i in keys if i>keys[idxKey])
    return dict((k, commonLocs[k]) for k in lookAheadKeys )

def featureCreation(locDict, idxKey):
    ''' gives out clean features and labels for a given locDict and a idxKey '''
    featuresIdxDirFileLoc= locDict[keys[idxKey]][0]
    labelsIdxDirFileLoc= locDict[keys[idxKey]][1]
    ''' read the features file'''
    featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='latin1') 
    ''' read the labels file'''
    labelsDf=pd.read_csv(labelsIdxDirFileLoc)
    ''' pop the labels out'''
    labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[ labelName])
    arrX = np.array(dfX)
    ''' feature normalisation'''
    #feature scaling in [0,1] - X = rescale_01(arrX)
    X = normalization(rescale_01(arrX))
    y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
    ''' returns features, labels'''
    return X, y
    

In [104]:
## testing above##
forwardDatesDict = forwardDates(idxKey = 1, commonLocs=commonLocs)
X, y = featureCreation(locDict = forwardDatesDict, idxKey =2)
print('checking shapes',X.shape[0]==y.shape[0])

checking shapes True


'20180201'

In [90]:

#for idxKey, commonDate in #enumerate(commonLocs.keys()):
for idxKey in range(3):
   
        print('Doing key:')
        print(keys[idxKey])
        ''' locations'''
        featuresIdxDirFileLoc= commonLocs[keys[idxKey]][0]
        labelsIdxDirFileLoc= commonLocs[keys[idxKey]][1]  

        featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='latin1')   
        ''' read the labels file'''
        labelsDf=pd.read_csv(labelsIdxDirFileLoc)
        ''' take in the input file'''
        dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                     featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
        #take the labels out
        labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 
        print ('preprocessing data...', end='')
        '''dataframe of Features and Labels - X and Y'''
        dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
        labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
        ''' drop the labels from the features'''
        dfX = dfXY.drop(columns=[ labelName])
        print("Shape of dfX..",dfX.shape[0])   
        ''' MKL stuff- convert the Dataframe into an array'''
        arrX = np.array(dfX)
        ''' feature normalisation'''
         #feature scaling in [0,1] - X = rescale_01(arrX)
        X = normalization(rescale_01(arrX))
        print (X.shape,'done')

        y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
        print("Shape of y..", y.shape)
        '''training and testing- this we can probably skip'''
        Xtr,Xte,Ytr,Yte = train_test_split(X,y, test_size=.1, random_state=42)

        '''compute homogeneous polynomial kernels with degrees 0,1,2,...,10'''

        print ('computing Homogeneous Polynomial Kernels...', end='')

        KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
        KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]

        ''' Compute RBF Kernels'''
        ### test set work ###
        #KLtest = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]
        gamma_range = np.logspace(-9, 3, 13)
        ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]

        print ('RBF Kernels done')

        print ('training AverageMKL...', end='')
        clf = AverageMKL().fit(KLtr,Ytr) #a wrapper for averaging kernels
   
        ''' need to serialise this--here'''
        print ('done')
        print(clf.weights) #print the weights of the combination of base kernels

        print ('training EasyMKL...for polynomials and RBF', end='')
        clfEasy = EasyMKL(lam=0.1).fit(KLtr,Ytr)#combining kernels with the EasyMKL algorithm
 
        clfRBF = EasyMKL(lam=0.1).fit(ker_list,Ytr)  
         
        #lam is a hyper-parameter in [0,1]
        print ('weights:')
        print (clfEasy.weights)
        print(clfRBF.weights)
        print('Training Done')
           
        
        ''' Now do the testing'''
        try: 
            print('Average Kernel Testing')
            y_pred = clf.predict(KLte)                 #predictions
            y_score = clf.decision_function(KLte)      #rank
            accuracy = accuracy_score(Yte, y_pred)
            roc_auc = roc_auc_score(Yte, y_score)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

            ''' Test Linear'''
            print('MKL-Linear Testing')
            y_predTest = clfEasy.predict(KLte)                 #predictions
            y_scoreTest = clfEasy.decision_function(KLte)      #rank
            accuracy = accuracy_score(Yte, y_predTest)
            roc_auc = roc_auc_score(Yte, y_scoreTest)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

            '''Test RBF'''
            print('MKL-RBF Testing')
            y_predRBF = clfRBF.predict(KLte)                 #predictions
            y_scoreRBF = clfRBF.decision_function(KLte)      #rank
            accuracyRBF = accuracy_score(Yte, y_predRBF)
            roc_aucRBF = roc_auc_score(Yte, y_scoreRBF)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracyRBF, roc_aucRBF))
        except IndexError: #  catch the error      
            continue# pass will basically ignore it

    
    

Doing key:
20170830
preprocessing data...Shape of dfX.. 2222
(2222, 21) done
Shape of y.. (2222,)
computing Homogeneous Polynomial Kernels...RBF Kernels done
training AverageMKL...done
[0.25 0.25 0.25 0.25]
training EasyMKL...for polynomials and RBFweights:
[-3.84941212e-24  8.54017201e-02  2.43016101e-01  6.71582179e-01]
[3.41964971e-12 3.42038312e-11 3.42039020e-10 3.42036239e-09
 3.42036593e-08 3.42042401e-07 3.42101009e-06 3.42683796e-05
 3.48197830e-04 3.79722345e-03 3.23094019e-02 1.69566547e-01
 7.93940560e-01]
Training Done
Average Kernel Testing
Accuracy score: 0.982, roc AUC score: 0.671
MKL-Linear Testing
Accuracy score: 0.857, roc AUC score: 0.776
MKL-RBF Testing
Accuracy score: 0.978, roc AUC score: 0.780
Doing key:
20170831
preprocessing data...Shape of dfX.. 2963
(2963, 21) done
Shape of y.. (2963,)
computing Homogeneous Polynomial Kernels...RBF Kernels done
training AverageMKL...done
[0.25 0.25 0.25 0.25]
training EasyMKL...for polynomials and RBFweights:
[-1.12793006e-

In [80]:
testPath = os.path.join(MKLSymbolPath, str(keys[idxKey])+'RBFMKL.pickle')


with open(testPath, 'rb') as handle:
    pickledCLF = pickle.load(handle)


SyntaxError: invalid syntax (<ipython-input-127-8f153a7bd83e>, line 1)

In [111]:
KLtr[1].shape

(4065, 4065)

### Test Set ###

In [ ]:
testKey=1
if testKey != idxKey:
    testFeaturesIdxDirFileLoc= commonLocs[keys[testKey]][0]
    testLabelsIdxDirFileLoc= commonLocs[keys[testKey]][1]
    testFeaturesTupleFile = pickle.load(open(testFeaturesIdxDirFileLoc,"rb"), encoding='latin1')   
    testLabelsDf=pd.read_csv(testLabelsIdxDirFileLoc)
else:
    print('you have same dates')

In [ ]:
testDirectory = os.getcwd()
# save the model to disk
filename = os.path.join(testDirectory,'test_MKL_model.pkl')
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model

### pick up the train features ###

In [ ]:

dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                 featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)

In [ ]:
#take the labels out
labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 

labelsShift = labels.isna().sum() # going to use this for "allignment of features and labels"
labelsShift

### pick up the test-features ###

In [ ]:

dfTestFeatures = pd.concat([testFeaturesTupleFile [0], testFeaturesTupleFile[1],\
                                                 testFeaturesTupleFile[2], testFeaturesTupleFile[3]], axis=1, sort=False).fillna(0)

In [ ]:
#take the labels out
testLabels =testLabelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 

testLabelsShift = testLabelsDf.isna().sum() # going to use this for "allignment of features and labels"


### put labels and features together before you dropnans -train/test###

In [ ]:

dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])

In [ ]:

dfTestXY = pd.concat([dfTestFeatures, testLabels], axis=1, sort='False').dropna()
testLabelName = str(dfTestXY.columns[dfTestXY.columns.str.contains(pat='label')].values[0])

## get working now on features and labels for MKL ##

In [ ]:
dfX = dfXY.drop(columns=[ labelName])
print("Shape of dfX..",dfX.shape[0])    
dfX.columns.values


In [ ]:
# dfTestX = dfTestXY.drop(columns=[ labelName])
# print("Shape of dfTestX..",dfTestX.shape[0])    
# dfTestX.columns.values==dfX.columns.values

## split the data + standarise ##

In [ ]:
X =StandardScaler().fit_transform(dfX)
y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(X.shape, y.shape)

In [ ]:
# testX = StandardScaler().fit_transform(dfTestXY)
# testY = dfTestXY[dfTestXY.columns[dfTestXY.columns.str.contains(pat='label')]].iloc[:, 0]

## Simple Classifiers for Comparison ##

In [ ]:
y

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
y_binarised = label_binarize(y, classes=[0, 1])
n_classes = y_binarised.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Yte[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Yte.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#    y_score = clf.decision_function(KLte)      #rank
#     accuracy = accuracy_score(Yte, y_pred)
#     roc_auc = roc_auc_score(Yte, y_score)

In [ ]:
fpr, tpr, thresholds =roc_curve(Yte, y_score)

roc_auc = auc(fpr, tpr)

fprRBF, tprRBF, thresholdsRBF =roc_curve(Yte, y_scoreRBF)

roc_aucRBF = auc(fprRBF, tprRBF)
print(roc_aucRBF)

In [ ]:
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot(fpr, tpr, color='darkred',)
plt.plot(fprRBF, tprRBF, color='darkorange',)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--',label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--',label='ROC curve (area = %0.2f)' % roc_aucRBF)
plt.legend(loc="lower left")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (MKL)')

plt.show()


In [ ]:
y_predRBF = clfRBF.predict(KLte)                 #predictions
y_scoreRBF = clfRBF.decision_function(KLte)      #rank
accuracyRBF = accuracy_score(Yte, y_predRBF)
roc_aucRBF = roc_auc_score(Yte, y_scoreRBF)

### Trying MKL for the very first time ###

In [ ]:
#preprocess data
print ('preprocessing data...', end='')
from MKLpy.preprocessing import normalization, rescale_01
arrX = np.array(dfX)
X = rescale_01(arrX) #feature scaling in [0,1]
X = normalization(rescale_01(arrX))
Y=y
print(X.shape, y.shape)
# ## test set
arrTestX = np.array(dfTestX)
TestX = rescale_01(arrTestX) #feature scaling in [0,1]
TestX = normalization(rescale_01(arrTestX))
TestY=testY

### splitting the date into training and testing ###

In [ ]:
Xtr,Xte,Ytr,Yte = train_test_split(X,Y, test_size=.55, random_state=42)
print (X.shape,'done')

In [ ]:
#compute homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing Homogeneous Polynomial Kernels...', end='')

from MKLpy.metrics import pairwise
KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]
from sklearn.metrics.pairwise import rbf_kernel
gamma_range = np.logspace(-9, 3, 13)
ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]
print ('RBF Kernels done')
### test set work ###
KLtest = [pairwise.homogeneous_polynomial_kernel(TestX,Xtr, degree=d) for d in range(4)]
from sklearn.metrics.pairwise import rbf_kernel
gamma_range = np.logspace(-9, 3, 13)
ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]


In [ ]:
#clf.fit(X_train, y_train)
classifier = SVC(kernel='linear', C=0.01)
linear_clf=classifier.fit(Xtr, Ytr)
# Create a SVC classifier using an RBF kernel
rbf_classifier= SVC(kernel='rbf', random_state=0, gamma=1000, C=1000)
rbf_clf=rbf_classifier.fit(Xtr, Ytr)

In [ ]:
#MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
print ('training AverageMKL...', end='')
clf = AverageMKL().fit(KLtr,Ytr) #a wrapper for averaging kernels
print ('done')
print(clf.weights) #print the weights of the combination of base kernels
K_average = clf.ker_matrix #the combined kernel matrix

In [ ]:
# clfRBF = clf = AverageMKL().fit(ker_list,Ytr) #a wrapper for averaging kernels
K_average_rbf = clfRBF.ker_matrix

In [ ]:
dfKaverage =pd.DataFrame(K_average)
dfKaverageRBF =pd.DataFrame(K_average_rbf)


In [ ]:

_=plt.matshow(dfKaverage, fignum=100)
plt.gca().set_aspect('auto')
plt.title(' K-average matrix' ,y=1.18)
plt.show()
plt.matshow(dfKaverageRBF, fignum=100)
plt.gca().set_aspect('auto')
_=plt.title('K-average RBF matrix', y=1.18)
plt.show()

In [ ]:
print ('training EasyMKL...', end='')
clf = EasyMKL(lam=0.1).fit(KLtr,Ytr)#combining kernels with the EasyMKL algorithm
clfEasyRBF = EasyMKL(lam=0.1).fit(ker_list,Ytr)
#lam is a hyper-parameter in [0,1]
print ('done')
print (clf.weights)

In [ ]:
_=plt.hist(clfEasyRBF.weights)
plt.show()
_=plt.hist (clf.weights)

In [ ]:
dfWeights =pd.DataFrame()

In [ ]:
dfweights = pd.DataFrame(clfEasyRBF.weights) # pd.DataFrame(dict(rate=np.random.randn(10000)))
dfweights2 = pd.DataFrame(clf.weights) #pd.DataFrame(dict(rate=np.random.randn(10000)))
dfweights.rename(columns={'0':'weights'})

fig, axes = plt.subplots(1, 2)

_=dfweights.hist( ax=axes[0],label='RBF')

_=dfweights2.hist( ax=axes[1])
plt.show()

In [ ]:
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 
y_pred = clf.predict(KLte)                 #predictions
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)
roc_auc = roc_auc_score(Yte, y_score)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
y_pred = clf.predict(KLte)                 #predictions
y_pred_linear = linear_clf.predict(Xte)
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)
roc_auc = roc_auc_score(Yte, y_score)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
y_predTest = clf.predict(KLtest)                 #predictions
y_scoreTest = clf.decision_function(KLtest)      #rank
accuracy = accuracy_score(TestY, y_predTest)
roc_auc = roc_auc_score(TestY, y_scoreTest)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
plt.plot(clf.decision_function(KLtr))

In [ ]:
# print ('tuning lambda for EasyMKL...', end='')
# base_learner = SVC(C=100)	#simil hard-margin svm
# best_results = {}
# for lam in [0, 0.01, 0.1, 0.2, 0.9, 1]:#possible lambda values for the EasyMKL algorithm
#     #MKLpy.model_selection.cross_val_predict performs the cross validation automatically, it optimizes the accuracy
#     #the counterpart cross_val_score optimized the roc_auc_score (use score='roc_auc')
#     #WARNING: these functions will change in the next version
#     scores = cross_val_predict(KLtr, Ytr, EasyMKL(learner=base_learner, lam=lam), n_folds=5, score='accuracy')
#     acc = np.mean(scores)
# #     if not best_results or best_results['score'] < acc:
# #         best_results = {'lam' : lam, 'score' : acc}